<a href="https://colab.research.google.com/github/Madhura0802/Vector-Addition/blob/main/Vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version






nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
code = """
#include <stdio.h>
#include <stdlib.h>
#define N 1000000
__global__ void add(int *a, int *b, int *c) {
int tid = blockIdx.x * blockDim.x + threadIdx.x;
if (tid < N) {
c[tid] = a[tid] + b[tid];
}
}
int main() {
int *a, *b, *c;
int *d_a, *d_b, *d_c;
int size = N * sizeof(int);
// Allocate memory on host
a = (int*)malloc(size);
b = (int*)malloc(size);
c = (int*)malloc(size);
// Initialize arrays
for (int i = 0; i < N; i++) {
a[i] = i;
b[i] = i * 2;
}
// Allocate memory on device
cudaMalloc(&d_a, size);
cudaMalloc(&d_b, size);
cudaMalloc(&d_c, size);
// Copy data from host to device
cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
// Launch kernel with 1 million threads
add<<<(N + 255) / 256, 256>>>(d_a, d_b, d_c);
// Copy result from device to host
cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
// Print first and last elements of result
printf("c[0]=%d, c[%d] = %d",c[0],N-1,c[N-1]);
// Free memory
free(a);
free(b);
free(c);
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}
"""


In [ ]:
text_file = open("Untitled2.cu","w")
text_file.write(code)
text_file.close()


In [ ]:
!nvcc "Untitled2.cu

/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [ ]:
!./a.out

c[0]=0, c[999999] = 2999997

In [ ]:
!nvprof ./a.out

==4256== NVPROF is profiling process 4256, command: ./a.out
==4256== c[0]=0, c[999999] = 2999997Profiling application: ./a.out
==4256== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   50.33%  1.5330ms         1  1.5330ms  1.5330ms  1.5330ms  [CUDA memcpy DtoH]
                   48.12%  1.4654ms         2  732.71us  695.15us  770.26us  [CUDA memcpy HtoD]
                    1.55%  47.200us         1  47.200us  47.200us  47.200us  add(int*, int*, int*)
      API calls:   97.09%  217.99ms         3  72.664ms  67.341us  217.84ms  cudaMalloc
                    2.07%  4.6534ms         3  1.5511ms  943.45us  2.7495ms  cudaMemcpy
                    0.52%  1.1681ms         1  1.1681ms  1.1681ms  1.1681ms  cuDeviceGetPCIBusId
                    0.23%  513.24us         3  171.08us  133.21us  196.75us  cudaFree
                    0.06%  127.49us       101  1.2620us     132ns  51.606us  cuDeviceGetAttribute
                